In [1]:
!git clone https://github.com/drryodino246/csc542-gp-team164.git

!pip install datasets

from datasets import load_dataset
emotions = load_dataset('csv', data_files='/content/csc542-gp-team164/baseline.csv')

emotions_split = emotions['train'].train_test_split(test_size=0.2, seed=42)
train_dataset = emotions_split['train']
test_dataset = emotions_split['test']

from transformers import AutoTokenizer

model_ckpt = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

def tokenize(batch):
  return tokenizer(batch["text"], padding=True, truncation=True, max_length=512, return_tensors="pt")

# Tokenize both the train and test datasets
train_encoded = train_dataset.map(tokenize, batched=True)
test_encoded = test_dataset.map(tokenize, batched=True)

# Rename 'updated_emotion' to 'label' for both train and test datasets
train_encoded = train_encoded.rename_column("updated_emotion", "label")
test_encoded = test_encoded.rename_column("updated_emotion", "label")

# Map string labels to integer labels
def map_labels(example):
    label_map = {
        "very negative": 0,
        "negative": 1,
        "positive": 2,
        "very positive": 3
    }
    example["label"] = label_map[example["label"]]  # Replace string label with integer label
    return example

# Apply the label mapping function to both train and test datasets
train_encoded = train_encoded.map(map_labels)
test_encoded = test_encoded.map(map_labels)

from transformers import AutoModelForSequenceClassification
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_labels = 4
id2label = {
    "0": "very negative",
    "1": "negative",
    "2": "positive",
    "3": "very positive"
}
label2id = {
    "very negative": 0,
    "negative": 1,
    "positive": 2,
    "very positive": 3
}

model = AutoModelForSequenceClassification.from_pretrained(model_ckpt, num_labels=num_labels, id2label=id2label, label2id=label2id).to(device)

!pip install tensorboard

from transformers import TrainingArguments
import tensorflow as tf
from torch.utils.tensorboard import SummaryWriter

batch_size = 32
logging_steps = 500
model_name = f"{model_ckpt}-finetuned-emotion"

training_args = TrainingArguments(
    output_dir=model_name,
    num_train_epochs=4,
    learning_rate=3e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir=None,  # Directory for logs
    logging_steps=logging_steps,  # Log training progress every 'logging_steps'
    load_best_model_at_end=False,  # Load the best model based on evaluation metrics
    metric_for_best_model="f1",  # Track the F1 score for selecting the best model
    greater_is_better=True,  # We want higher F1 score, so set this to True
    disable_tqdm=False,  # Keep TQDM for progress bar
    report_to="none",  # Enable TensorBoard logging
)

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import numpy as np

def compute_metrics(p):
    preds, labels = p
    preds = np.argmax(preds, axis=1)

    accuracy = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average='weighted')
    precision = precision_score(labels, preds, average='weighted')
    recall = recall_score(labels, preds, average='weighted')

    return {
        'accuracy': accuracy,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_encoded,
    eval_dataset=test_encoded,
    tokenizer=tokenizer
)

trainer.train()
#trainer.evaluate()

Cloning into 'csc542-gp-team164'...
remote: Enumerating objects: 82, done.
remote: Counting objects: 100% (82/82), done.
remote: Compressing objects: 100% (72/72), done.
remote: Total 82 (delta 48), reused 24 (delta 8), pack-reused 0 (from 0)
Receiving objects: 100% (82/82), 701.64 KiB | 20.64 MiB/s, done.
Resolving deltas: 100% (48/48), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages t

Generating train split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<ipython-input-1-7539949ede34>:112: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.723244,0.705760,0.703452,0.708164,0.705760
2,0.846900,0.723966,0.716966,0.717386,0.725960,0.716966
3,0.571200,0.727785,0.733646,0.733075,0.733505,0.733646
4,0.409300,0.773551,0.729476,0.729070,0.728884,0.729476


TrainOutput(global_step=1920, training_loss=0.545414924621582, metrics={'train_runtime': 706.1259, 'train_samples_per_second': 86.936, 'train_steps_per_second': 2.719, 'total_flos': 1978209064693512.0, 'train_loss': 0.545414924621582, 'epoch': 4.0})